In [132]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [133]:
%run verifica.ipynb

# Loading Features

### Datasets with low, medium and high standard deviation

In [134]:
low_2d_sd= pd.read_csv("data_0.5.csv") # Read the dataset with lowest standard deviation
medium_2d_sd= pd.read_csv("data_1.0.csv") # Read the dataset with medium standard deviation
high_2d_sd= pd.read_csv("data_1.0.csv") # Read the dataset with highest standard deviation

In [135]:
# Checking column names
low_2d_sd.head()

,Nodule_id,Patient_id,Subtlety,Internalstructure,Calcification,Sphericity,Margin,Lobulation,Spiculation,Texture,Malignancy,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,diagnostics_Mask-original_Spacing,diagnostics_Mask-original_Size,diagnostics_Mask-original_BoundingBox,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Mask-original_CenterOfMassIndex,diagnostics_Mask-original_CenterOfMass,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,original_firstorder_MeanAbsoluteDeviation,original_firstorder_Mean,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RobustMeanAbsoluteDeviation,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_TotalEnergy,original_firstorder_Uniformity,original_firstorder_Variance,original_glcm_Autocorrelation,original_glcm_JointAverage,original_glcm_ClusterProminence,original_glcm_ClusterShade,original_glcm_ClusterTendency,original_glcm_Contrast,original_glcm_Correlation,original_glcm_DifferenceAverage,original_glcm_DifferenceEntropy,original_glcm_DifferenceVariance,original_glcm_JointEnergy,original_glcm_JointEntropy,original_glcm_Imc1,original_glcm_Imc2,original_glcm_Idm,original_glcm_Idmn,original_glcm_Id,original_glcm_Idn,original_glcm_InverseVariance,original_glcm_MaximumProbability,original_glcm_SumEntropy,original_glcm_SumSquares,original_glrlm_GrayLevelNonUniformity,original_glrlm_GrayLevelNonUniformityNormalized,original_glrlm_GrayLevelVariance,original_glrlm_HighGrayLevelRunEmphasis,original_glrlm_LongRunEmphasis,original_glrlm_LongRunHighGrayLevelEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glrlm_LowGrayLevelRunEmphasis,original_glrlm_RunEntropy,original_glrlm_RunLengthNonUniformity,original_glrlm_RunLengthNonUniformityNormalized,original_glrlm_RunPercentage,original_glrlm_RunVariance,original_glrlm_ShortRunEmphasis,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glrlm_ShortRunLowGrayLevelEmphasis,original_glszm_GrayLevelNonUniformity,original_glszm_GrayLevelNonUniformityNormalized,original_glszm_GrayLevelVariance,original_glszm_HighGrayLevelZoneEmphasis,original_glszm_LargeAreaEmphasis,original_glszm_LargeAreaHighGrayLevelEmphasis,original_glszm_LargeAreaLowGrayLevelEmphasis,original_glszm_LowGrayLevelZoneEmphasis,original_glszm_SizeZoneNonUniformity,original_glszm_SizeZoneNonUniformityNormalized,original_glszm_SmallAreaEmphasis,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_gldm_DependenceEntropy,original_gldm_DependenceNonUniformity,original_gldm_DependenceNonUniformityNormalized,original_gldm_DependenceVariance,original_gldm_GrayLevelNonUniformity,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighG

### Renaming the columns


In [136]:
# Mapping the new column names to the existing columns in a dictionary format
new_column_names = {'Nodule 1\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n': 'Diagnosis Nodule 1', 
                    'Nodule 1\nDiagnosis Method at the Nodule Level\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response\n': 'Diagnosis Method Nodule 1',
                      'Nodule 2\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n': 'Diagnosis Nodule 2',
                      'Nodule 2\nDiagnosis Method at the Nodule Level\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response\n': 'Diagnosis Method Nodule 2',
                      'Nodule 3\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n': 'Diagnosis Nodule 3',
                      'Nodule 3\nDiagnosis Method at the Nodule Level\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response\n': 'Diagnosis Method Nodule 3',
                      'Nodule 4\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n': 'Diagnosis Nodule 4',
                      'Nodule 4\nDiagnosis Method at the Nodule Level\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response\n': 'Diagnosis Method Nodule 4',
                      'Nodule 5\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n': 'Diagnosis Nodule 5',
                      'Nodule 5\nDiagnosis Method at the Nodule Level\n0 = unknown\n1 = review of radiological images to show 2 years of stable nodule\n2 = biopsy\n3 = surgical resection\n4 = progression or response\n': 'Diagnosis Method Nodule 5'}
# Renaming columns in DataFrames
low_2d_sd.rename(columns=new_column_names, inplace=True)
medium_2d_sd.rename(columns=new_column_names, inplace=True)
high_2d_sd.rename(columns=new_column_names, inplace=True)

## Processing Data

### Handling Missing Data:

In [137]:
# Check if missing data exists
missing_data(low_2d_sd)

Missing values in Diagnosis at the Patient Level
0=Unknown
1=benign or non-malignant disease
2= malignant, primary lung cancer
3 = malignant metastatic
: 847
Missing values in Diagnosis Method
0 = unknown
1 = review of radiological images to show 2 years of stable nodule
2 = biopsy
3 = surgical resection
4 = progression or response: 847
Missing values in Primary tumor site for metastatic disease: 847
Missing values in Diagnosis Nodule 1: 865
Missing values in Diagnosis Method Nodule 1: 865
Missing values in Diagnosis Nodule 2: 954
Missing values in Diagnosis Method Nodule 2: 954
Missing values in Diagnosis Nodule 3: 966
Missing values in Diagnosis Method Nodule 3: 966
Missing values in Diagnosis Nodule 4: 966
Missing values in Diagnosis Method Nodule 4: 966
Missing values in Diagnosis Nodule 5: 966
Missing values in Diagnosis Method Nodule 5: 966


In [138]:
# Check if missing data exists
missing_data(medium_2d_sd)

Missing values in Diagnosis at the Patient Level
0=Unknown
1=benign or non-malignant disease
2= malignant, primary lung cancer
3 = malignant metastatic
: 1416
Missing values in Diagnosis Method
0 = unknown
1 = review of radiological images to show 2 years of stable nodule
2 = biopsy
3 = surgical resection
4 = progression or response: 1416
Missing values in Primary tumor site for metastatic disease: 1416
Missing values in Diagnosis Nodule 1: 1448
Missing values in Diagnosis Method Nodule 1: 1448
Missing values in Diagnosis Nodule 2: 1627
Missing values in Diagnosis Method Nodule 2: 1627
Missing values in Diagnosis Nodule 3: 1655
Missing values in Diagnosis Method Nodule 3: 1655
Missing values in Diagnosis Nodule 4: 1656
Missing values in Diagnosis Method Nodule 4: 1656
Missing values in Diagnosis Nodule 5: 1656
Missing values in Diagnosis Method Nodule 5: 1656


In [139]:
# Check if missing data exists
missing_data(high_2d_sd)

Missing values in Diagnosis at the Patient Level
0=Unknown
1=benign or non-malignant disease
2= malignant, primary lung cancer
3 = malignant metastatic
: 1416
Missing values in Diagnosis Method
0 = unknown
1 = review of radiological images to show 2 years of stable nodule
2 = biopsy
3 = surgical resection
4 = progression or response: 1416
Missing values in Primary tumor site for metastatic disease: 1416
Missing values in Diagnosis Nodule 1: 1448
Missing values in Diagnosis Method Nodule 1: 1448
Missing values in Diagnosis Nodule 2: 1627
Missing values in Diagnosis Method Nodule 2: 1627
Missing values in Diagnosis Nodule 3: 1655
Missing values in Diagnosis Method Nodule 3: 1655
Missing values in Diagnosis Nodule 4: 1656
Missing values in Diagnosis Method Nodule 4: 1656
Missing values in Diagnosis Nodule 5: 1656
Missing values in Diagnosis Method Nodule 5: 1656


In [140]:
# Filtering NaN values in 'Diagnosis at the Patient Level' column
low_2d_sd = low_2d_sd.dropna(subset=['Diagnosis at the Patient Level\n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic\n'])
medium_2d_sd = low_2d_sd.dropna(subset=['Diagnosis at the Patient Level\n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic\n'])
high_2d_sd = low_2d_sd.dropna(subset=['Diagnosis at the Patient Level\n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic\n'])

In [141]:
# Creating a boolean DataFrame to identify missing (NaN) values in low_2d_sd DataFrame
missing_values = low_2d_sd.isna()

# Stacking the missing_values DataFrame to get a Series of boolean values indicating missing data
missing_indices = missing_values.stack()

# Generating a dictionary containing columns with missing values and their respective indices
missing_values_dict = find_missing_values(low_2d_sd)
missing_values_dict1 = find_missing_values(medium_2d_sd)
missing_values_dict2 = find_missing_values(high_2d_sd)

### Replacing NaN values with zeros

In [142]:
# Replacing NaN values with zero in the 'low_2d_sd' DataFrame
low_2d_sd = replace_nan_with_zero(low_2d_sd)

# Replacing NaN values with zero in the 'medium_2d_sd' DataFrame
medium_2d_sd = replace_nan_with_zero(medium_2d_sd)

# Replacing NaN values with zero in the 'high_2d_sd' DataFrame
high_2d_sd = replace_nan_with_zero(high_2d_sd)

### Checking if there are still missing values for all datasets

In [143]:
for column, missing_rows in missing_values_dict.items():
    print(f"Column with missing values: {column}")
    print(f"Rows with missing values in this column: {missing_rows}")
    print()

Column with missing values: Diagnosis Nodule 1
Rows with missing values in this column: [330, 331, 362, 429, 430, 431, 432, 537, 538, 539, 540, 541, 542, 837, 853, 854, 876, 878]

Column with missing values: Diagnosis Method Nodule 1
Rows with missing values in this column: [330, 331, 362, 429, 430, 431, 432, 537, 538, 539, 540, 541, 542, 837, 853, 854, 876, 878]

Column with missing values: Diagnosis Nodule 2
Rows with missing values in this column: [66, 80, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 117, 118, 129, 130, 141, 142, 143, 144, 145, 146, 149, 150, 151, 152, 153, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 185, 193, 194, 198, 199, 200, 201, 202, 203, 204, 210, 211, 212, 213, 214, 216, 217, 218, 219, 220, 221, 223, 297, 301, 326, 330, 331, 362, 429, 430, 431, 432, 479, 537, 538, 539, 540, 541, 542, 837, 853, 854, 876, 878, 889, 890, 935, 936, 937, 953, 964]

Column wit

In [144]:
for column, missing_rows in missing_values_dict1.items():
    print(f"Column with missing values: {column}")
    print(f"Rows with missing values in this column: {missing_rows}")
    print()

Column with missing values: Diagnosis Nodule 1
Rows with missing values in this column: [330, 331, 362, 429, 430, 431, 432, 537, 538, 539, 540, 541, 542, 837, 853, 854, 876, 878]

Column with missing values: Diagnosis Method Nodule 1
Rows with missing values in this column: [330, 331, 362, 429, 430, 431, 432, 537, 538, 539, 540, 541, 542, 837, 853, 854, 876, 878]

Column with missing values: Diagnosis Nodule 2
Rows with missing values in this column: [66, 80, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 117, 118, 129, 130, 141, 142, 143, 144, 145, 146, 149, 150, 151, 152, 153, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 185, 193, 194, 198, 199, 200, 201, 202, 203, 204, 210, 211, 212, 213, 214, 216, 217, 218, 219, 220, 221, 223, 297, 301, 326, 330, 331, 362, 429, 430, 431, 432, 479, 537, 538, 539, 540, 541, 542, 837, 853, 854, 876, 878, 889, 890, 935, 936, 937, 953, 964]

Column wit

In [145]:
for column, missing_rows in missing_values_dict2.items():
    print(f"Column with missing values: {column}")
    print(f"Rows with missing values in this column: {missing_rows}")
    print()

Column with missing values: Diagnosis Nodule 1
Rows with missing values in this column: [330, 331, 362, 429, 430, 431, 432, 537, 538, 539, 540, 541, 542, 837, 853, 854, 876, 878]

Column with missing values: Diagnosis Method Nodule 1
Rows with missing values in this column: [330, 331, 362, 429, 430, 431, 432, 537, 538, 539, 540, 541, 542, 837, 853, 854, 876, 878]

Column with missing values: Diagnosis Nodule 2
Rows with missing values in this column: [66, 80, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 117, 118, 129, 130, 141, 142, 143, 144, 145, 146, 149, 150, 151, 152, 153, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 185, 193, 194, 198, 199, 200, 201, 202, 203, 204, 210, 211, 212, 213, 214, 216, 217, 218, 219, 220, 221, 223, 297, 301, 326, 330, 331, 362, 429, 430, 431, 432, 479, 537, 538, 539, 540, 541, 542, 837, 853, 854, 876, 878, 889, 890, 935, 936, 937, 953, 964]

Column wit

## Encoding Categorical Data¶

### Mapping non-numeric values to numeric values in 'Malignancy' column

In [146]:
malignancy_mapping = {
    '1-Highly Unlikely': 1,
    '2-Moderately Unlikely': 2,
    '3-Indeterminate': 3,
    '4-Moderately Suspicious': 4,
    '5-Highly Suspicious': 5
}

low_2d_sd['Malignancy'] = low_2d_sd['Malignancy'].map(malignancy_mapping)
medium_2d_sd['Malignancy'] = medium_2d_sd['Malignancy'].map(malignancy_mapping)
high_2d_sd['Malignancy'] = high_2d_sd['Malignancy'].map(malignancy_mapping)

### Uniforming the labels

In [147]:
labels = [
    'Head, Neck', 'Carcinoma', 'Prostate', 'Prostate', 'Prostate', 'Colon', 'Colon ', 'Colon', 'Colon', 'Colon',
    'Colon', 'Colon', 'Colon', 'Colon', 'Colon', 'Colon', 'Carcinoma', 'Lung', 'Sarcoma', 'Sarcoma',
    'Sarcoma', 'Sarcoma', 'Sarcoma', 'Sarcoma', 'Pancreatic', 'Gallbladder', 'Colorectal', 'Colorectal', 'Vaginal', 'Lung',
    'Lymphoma', 'Lymphoma', 'Lymphoma', 'Gallbladder', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung',
    'Lung', 'Lymphoma', 'Carcinoma', 'Carcinoma', 'Lung', 'Lung', 'Lung', 'Lung', 'Leukemia', 'Leukemia','Ovarian',
    'Ovarian', 'Ovarian', 'Ovarian', 'Rectal', 'Rectal', 'Rectal', 'Rectal', 'Sarcoma', 'Melanoma', 'Lung', 'Lung',
    'Breast', 'Breast', 'Rectal', 'Cervix', 'Leukemia', 'Leukemia', 'Sarcoma', 'Prostate', 'Prostate ', 'Laryngeal',
    'Multiple Primary Cancers', 'Lung', 'Lung', 'Sarcoma', 'Head, Neck', 'Esophageal', 'Leukemia', 'Melanoma', 'Lymphoma', 'Sarcoma',
    'Renal', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung', 'Breast', 'Cervical', 'Mesothelial', 'Mesothelial', 'Carcinoma', 'Lung', 'Fibrosis', 'Fibrosis', 'Fibrosis',
    'Fibrosis', 'Lung', 'Histoplasmosis', 'Histoplasmosis', 'Histoplasmosis', 'Histoplasmosis', 'Histoplasmosis', 'Histoplasmosis', 'Carcinoma', 'Metastatic',
    'Metastatic', 'Carcinoma', 'Carcinoma', 'Lung', 'Lung', 'Lung', 'Lung', 'Lung','Lung','Lymphoma', 'Carcinoma'
]

# Create a LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels
encoded_labels = label_encoder.fit_transform(labels)

# Substitute the labels in the datasets
low_2d_sd['Primary tumor site for metastatic disease'] = encoded_labels
medium_2d_sd['Primary tumor site for metastatic disease'] = encoded_labels
high_2d_sd['Primary tumor site for metastatic disease'] = encoded_labels

### Removing patients prefixes (to have an int value)

In [148]:
remove_prefix(low_2d_sd, 'Patient_id', 'LIDC-IDRI-') 
low_2d_sd['Patient_id'] = pd.to_numeric(low_2d_sd['Patient_id'])

remove_prefix(medium_2d_sd, 'Patient_id', 'LIDC-IDRI-')
medium_2d_sd['Patient_id'] = pd.to_numeric(medium_2d_sd['Patient_id'])
                                           
remove_prefix(high_2d_sd, 'Patient_id', 'LIDC-IDRI-')
high_2d_sd['Patient_id'] = pd.to_numeric(high_2d_sd['Patient_id'])

### Drop unnecessary columns

In [149]:
columns_to_remove=['TCIA Patient ID', 'diagnostics_Configuration_EnabledImageTypes',
'diagnostics_Configuration_Settings',
'diagnostics_Image-original_Dimensionality',
'diagnostics_Image-original_Hash',
'diagnostics_Image-original_Maximum',
'diagnostics_Image-original_Mean',
'diagnostics_Image-original_Minimum',
'diagnostics_Image-original_Size',
'diagnostics_Image-original_Spacing',
'diagnostics_Mask-original_BoundingBox',
'diagnostics_Mask-original_CenterOfMass',
'diagnostics_Mask-original_CenterOfMassIndex',
'diagnostics_Mask-original_Hash',
'diagnostics_Mask-original_Size',
'diagnostics_Mask-original_Spacing',
'diagnostics_Mask-original_VolumeNum',
'diagnostics_Mask-original_VoxelNum',
'diagnostics_Versions_Numpy',
'diagnostics_Versions_PyRadiomics',
'diagnostics_Versions_PyWavelet',
'diagnostics_Versions_Python',
'diagnostics_Versions_SimpleITK', 'original_shape_Maximum3DDiameter',
'original_shape_MeshVolume']

low_2d_sd = low_2d_sd.drop(columns=columns_to_remove, errors='ignore')
medium_2d_sd = medium_2d_sd.drop(columns=columns_to_remove, errors='ignore')
high_2d_sd = high_2d_sd.drop(columns=columns_to_remove, errors='ignore')

### Check if there are stll string columns

In [150]:
string_columns = get_string_columns(low_2d_sd)  
print("Columns with string values:", string_columns)

Columns with string values: Index([], dtype='object')


### Remove unknown rows

In [151]:
# Check if any columns have all zeros
columns_with_zeros = (low_2d_sd == 0).all(axis=0)

# Get the column names with all zeros
columns_with_zeros = low_2d_sd.columns[columns_with_zeros]

if not columns_with_zeros.empty:
    print("Columns with all zeros (representing 'unknown'):")
    print(columns_with_zeros)
else:
    print("No columns with all zeros found.")

Columns with all zeros (representing 'unknown'):
Index(['original_firstorder_InterquartileRange',
       'original_firstorder_Kurtosis',
       'original_firstorder_MeanAbsoluteDeviation',
       'original_firstorder_Range',
       'original_firstorder_RobustMeanAbsoluteDeviation',
       'original_firstorder_Skewness', 'original_firstorder_Variance',
       'original_glcm_ClusterProminence', 'original_glcm_ClusterShade',
       'original_glcm_ClusterTendency', 'original_glcm_Contrast',
       'original_glcm_DifferenceAverage', 'original_glcm_DifferenceVariance',
       'original_glcm_Imc1', 'original_glcm_Imc2',
       'original_glcm_InverseVariance', 'original_glcm_SumSquares',
       'original_glrlm_GrayLevelVariance', 'original_glszm_GrayLevelVariance',
       'original_gldm_GrayLevelVariance', 'Diagnosis Nodule 3',
       'Diagnosis Method Nodule 3', 'Diagnosis Nodule 4',
       'Diagnosis Method Nodule 4', 'Diagnosis Nodule 5',
       'Diagnosis Method Nodule 5'],
      dtype='ob

In [152]:
list = ['original_firstorder_InterquartileRange',
       'original_firstorder_Kurtosis',
       'original_firstorder_MeanAbsoluteDeviation',
       'original_firstorder_Range',
       'original_firstorder_RobustMeanAbsoluteDeviation',
       'original_firstorder_Skewness', 'original_firstorder_Variance',
       'original_glcm_ClusterProminence', 'original_glcm_ClusterShade',
       'original_glcm_ClusterTendency', 'original_glcm_Contrast',
       'original_glcm_DifferenceAverage', 'original_glcm_DifferenceVariance',
       'original_glcm_Imc1', 'original_glcm_Imc2',
       'original_glcm_InverseVariance', 'original_glcm_SumSquares',
       'original_glrlm_GrayLevelVariance', 'original_glszm_GrayLevelVariance',
       'original_gldm_GrayLevelVariance']

low_2d_sd = low_2d_sd.drop(columns=list)
medium_2d_sd = medium_2d_sd.drop(columns=list)
high_2d_sd = high_2d_sd.drop(columns=list)